In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score , precision_score, recall_score

In [ ]:
#check the gpu is available for training
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
dataset_dir = "D:/finalyearproject/ricediseasedetection/rice_datatset/train"
test_dataset_dir = "D:/finalyearproject/ricediseasedetection/rice_datatset/test"
val_dataset_dir = "D:/finalyearproject/ricediseasedetection/rice_datatset/val"

In [ ]:
image_size = 224 #299 or 331
batch_size = 8
channel = 3
n_classes = 12

In [5]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    image_size=(image_size, image_size),
    batch_size=batch_size
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_dir,
    shuffle=True,
    image_size=(image_size, image_size),
    batch_size=batch_size
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dataset_dir,
    shuffle=True,
    image_size=(image_size, image_size),
    batch_size=batch_size
)

Found 6740 files belonging to 12 classes.
Found 2250 files belonging to 12 classes.
Found 2254 files belonging to 12 classes.


In [6]:
class_names = train_dataset.class_names
class_names

['bacterial_leaf_blight',
 'bacterial_leaf_streak',
 'bacterial_panicle_blight',
 'blast',
 'brown_spot',
 'dead_heart',
 'downy_mildew',
 'hispa',
 'normal',
 'rice_sheath_blight',
 'smut',
 'tungro']

In [ ]:
for image_batch,label_batch in train_dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype("uint8"))
    plt.title(class_names[label_batch[0]])
    plt.axis("off")

In [7]:
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
len(train_dataset)

843

In [9]:
len(test_dataset)

282

In [10]:
len(val_dataset)

282

In [12]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
    layers.RandomBrightness(factor=0.2),
    layers.RandomContrast(factor=0.2)
])

In [13]:
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size, image_size, channel)
)

In [15]:
base_model.trainable = False

In [17]:
model = models.Sequential([

    layers.Rescaling(1./255),

    base_model,

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(n_classes, activation='softmax')
])

In [18]:
model.build(input_shape=(None,image_size,image_size,channel))

In [19]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)              

                                                                 
 dense_5 (Dense)             (None, 12)                3084      
                                                                 
Total params: 27,694,668
Trainable params: 12,979,980
Non-trainable params: 14,714,688
_________________________________________________________________


In [20]:
lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=2,
    verbose=1,
)
early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=batch_size,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[lr_reducer, early_stopper]
)

In [ ]:
scores = model.evaluate(test_dataset)
print("Test Accuracy:", scores[1])

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def predict(model, image):
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 10))
for images, labels in test_dataset.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {confidence}%")
        plt.axis("off")
plt.tight_layout()

In [ ]:
y_true = []
y_pred = []
for x, y in test_dataset:
    y_pred.extend(np.argmax(model.predict(x), axis=1))
    y_true.extend(y)

cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

display_labels = [
    'bacterial_leaf_blight',
    'bacterial_leaf_streak',
    'bacterial_panicle_blight',
    'blast',
    'brown_spot',
    'dead_heart',
    'downy_mildew',
    'hispa',
    'normal',
    'rice_sheath_blight',
    'smut',
    'tungro'
]

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
f1_scores = f1_score(y_true, y_pred, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

print("F1 Scores:")
print("bacterial_leaf_blight:", f1_scores[0])
print("bacterial_leaf_streak:", f1_scores[1])
print("bacterial_panicle_blight:", f1_scores[2])
print("blast:", f1_scores[3])
print("brown_spot:", f1_scores[4])
print("dead_heart:", f1_scores[5])
print("downy_mildew:", f1_scores[6])
print("hispa:", f1_scores[7])
print("normal:", f1_scores[8])
print("rice_sheath_blight:", f1_scores[9])
print("smut:", f1_scores[10])
print("tungro:", f1_scores[11])

macro_f1 = f1_score(y_true, y_pred, average='macro')
print("Macro-averaged F1 Score:", macro_f1)

In [ ]:

precision = precision_score(y_true, y_pred, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

recall = recall_score(y_true, y_pred, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

print("Precision:")
print("bacterial_leaf_blight:", precision[0])
print("bacterial_leaf_streak:", precision[1])
print("bacterial_panicle_blight:", precision[2])
print("blast:", precision[3])
print("brown_spot:", precision[4])
print("dead_heart:", precision[5])
print("downy_mildew:", precision[6])
print("hispa:", precision[7])
print("normal:", precision[8])
print("rice_sheath_blight:", precision[9])
print("smut:", precision[10])
print("tungro:", precision[11])

print("\nRecall:")
print("bacterial_leaf_blight:", recall[0])
print("bacterial_leaf_streak:", recall[1])
print("bacterial_panicle_blight:", recall[2])
print("blast:", recall[3])
print("brown_spot:", recall[4])
print("dead_heart:", recall[5])
print("downy_mildew:", recall[6])
print("hispa:", recall[7])
print("normal:", recall[8])
print("rice_sheath_blight:", recall[9])
print("smut:", recall[10])
print("tungro:", recall[11])

In [ ]:
model.save('ricedisease_vgg16.h5')

In [ ]:
model.save('ricedisease_vgg16.keras')